# 环境

In [2]:
!pip install pydgraph

You should consider upgrading via the 'd:\software\python3.7\python.exe -m pip install --upgrade pip' command.


In [2]:
import requests
import json
import time
import pprint
import pydgraph

headers = {
    'Content-Type': 'application/json',
}

URL_PREFIX = 'http://localhost'
PORT_RATEL_HTTP = 8008
PORT_ALPHA_HTTP = 8088
PORT_ALPHA_GRPC = 9088

URL_PREFIX_GRAPHQL = URL_PREFIX + ':{port}/graphql'.format(
    port=PORT_ALPHA_HTTP)
URL_PREFIX_GRAPHQL_ADMIN = URL_PREFIX + ':{port}/admin'.format(
    port=PORT_ALPHA_HTTP)
print(URL_PREFIX_GRAPHQL)
print(URL_PREFIX_GRAPHQL_ADMIN)

http://localhost:8088/graphql
http://localhost:8088/admin


# /graphql, /admin


`/admin`:

- The  `health` query lets you know if everything is connected and if there’s a schema currently being served at /graphql.
- The `state` query returns the current state of the cluster and group membership information. For more information about state see here.
- The `config` query returns the configuration options of the cluster set at the time of starting it.
- The `getGQLSchema` query gets the current GraphQL schema served at /graphql, or returns null if there’s no such schema.
- The `getAllowedCORSOrigins` query returns your CORS policy.
- The `updateGQLSchema` mutation allows you to change the schema currently served at /graphql.


## Query schema

In [57]:
url = URL_PREFIX_GRAPHQL_ADMIN
print(url)

query = """
query {
  queryCustomer(filter: {username: {regexp: "/Mich.*/"}}) {
    username
    reviews(order: {asc: rating}, first: 5) {
      comment
      rating
      about {
        name
      }
    }
  }
}
"""
params = {
    'query': query
}

r = requests.get(url, params=params)
print(r)
pprint.pprint(r.json())

http://localhost:8088/admin
<Response [200]>
{'data': {'getGQLSchema': {'generatedSchema': '#######################\n'
                                              '# Input Schema\n'
                                              '#######################\n'
                                              '\n'
                                              'type Person {\n'
                                              '\tname: String\n'
                                              '}\n'
                                              '\n'
                                              '#######################\n'
                                              '# Extended Definitions\n'
                                              '#######################\n'
                                              '\n'
                                              '"""\n'
                                              'The Int64 scalar type '
                                              'represents a si

In [46]:
url = URL_PREFIX_GRAPHQL_ADMIN
print(url)

query = """
query {
          querySchemaHistory ( first : 10 ){
              schema
              created_at
           }
}
"""
params = {
    'query': query
}

r = requests.get(url, params=params)
print(r)
pprint.pprint(r.json())

http://localhost:8088/admin
<Response [200]>
{'data': {'querySchemaHistory': []},
 'extensions': {'tracing': {'duration': 5522600,
                            'endTime': '2021-02-03T10:37:29.0208931Z',
                            'execution': {'resolvers': [{'dgraph': [{'duration': 5207300,
                                                                     'label': 'query',
                                                                     'startOffset': 243100}],
                                                         'duration': 5321400,
                                                         'fieldName': 'querySchemaHistory',
                                                         'parentType': 'Query',
                                                         'path': ['querySchemaHistory'],
                                                         'returnType': '[SchemaHistory]',
                                                         'startOffset': 180400}]},
               

## Validate schema

In [53]:
url = URL_PREFIX_GRAPHQL_ADMIN + '/schema/validate'
print(url)

schema = """
type Product {
    productID: ID!
    name: String @search(by: [term])
    reviews: [Review] @hasInverse(field: about)
}

type Customer {
    username: String! @id @search(by: [hash, regexp])
    reviews: [Review] @hasInverse(field: by)
}

type Review {
    id: ID!
    about: Product!
    by: Customer!
    comment: String @search(by: [fulltext])
    rating: Int @search
}
"""

r = requests.post(url, data=schema)
print(r)
pprint.pprint(r.json())

http://localhost:8088/admin/schema/validate
<Response [200]>
{'errors': [{'extensions': {'code': 'success'}, 'message': 'Schema is valid'}]}


## Create schema

### post schema file

In [14]:
url = URL_PREFIX_GRAPHQL_ADMIN + '/schema'
print(url)

schema = """
type Product {
    productID: ID!
    name: String @search(by: [term])
    reviews: [Review] @hasInverse(field: about)
}

type Customer {
    username: String! @id @search(by: [hash, regexp])
    reviews: [Review] @hasInverse(field: by)
}

type Review {
    id: ID!
    about: Product!
    by: Customer!
    comment: String @search(by: [fulltext])
    rating: Int @search
}
"""
print(schema)

schema_content = schema.replace('\n', ' ')
print('|'+schema_content+'|')

files = {
    'file': ('products.graphql', schema_content, 'application/x-www-form-urlencoded'),
}
r = requests.post(url, files=files)
print(r.text)

print(r)
pprint.pprint(r.json())

http://localhost:8088/admin/schema

type Product {
    productID: ID!
    name: String @search(by: [term])
    reviews: [Review] @hasInverse(field: about)
}

type Customer {
    username: String! @id @search(by: [hash, regexp])
    reviews: [Review] @hasInverse(field: by)
}

type Review {
    id: ID!
    about: Product!
    by: Customer!
    comment: String @search(by: [fulltext])
    rating: Int @search
}

| type Product {     productID: ID!     name: String @search(by: [term])     reviews: [Review] @hasInverse(field: about) }  type Customer {     username: String! @id @search(by: [hash, regexp])     reviews: [Review] @hasInverse(field: by) }  type Review {     id: ID!     about: Product!     by: Customer!     comment: String @search(by: [fulltext])     rating: Int @search } |
{"errors":[{"message":"resolving updateGQLSchema failed because input:1: Unexpected \u003cInvalid\u003e\n (Locations: [{Line: 3, Column: 4}])","extensions":{"code":"Error"}}]}
<Response [200]>
{'errors': [{'exte

### updateGQLSchema

In [88]:
url = URL_PREFIX_GRAPHQL_ADMIN
print(url)

schema = """
 type Product {
    productID: ID!
    name: String @search(by: [term])
    reviews: [Review] @hasInverse(field: about)
}

 type Customer {
    username: String! @id @search(by: [hash, regexp])
    reviews: [Review] @hasInverse(field: by)
}

 type Review {
    id: ID!
    about: Product!
    by: Customer!
    comment: String @search(by: [fulltext])
    rating: Int @search
}
"""
print(schema)

schema_content = schema.replace('\n', '')
print(schema_content)
print()
mutation = """
mutation {
  updateGQLSchema( input: { set: { schema: """ + '"' + schema_content + '"' + """}})
  {
    gqlSchema {
      schema
      generatedSchema
    }
  }
}
""".replace('\n', '')

print(mutation)
print()

params = {}
data = {
    'query': mutation,
   # """mutation {  updateGQLSchema(    input: { set: { schema: "type Product {  productID: ID!  name: String @search(by: [term]) reviews: [Review] @hasInverse(field: about)}  type Customer { username: String! @id @search(by: [hash, regexp]) reviews: [Review] @hasInverse(field: by)}  type Review { id: ID! about: Product! by: Customer! comment: String @search(by: [fulltext]) rating: Int @search}"}})  {    gqlSchema {      schema      generatedSchema    }  }}"""
}
r = requests.post(url, headers=headers, params=params, data=json.dumps(data))

print(r)
pprint.pprint(r.json())

http://localhost:8088/admin

 type Product {
    productID: ID!
    name: String @search(by: [term])
    reviews: [Review] @hasInverse(field: about)
}

 type Customer {
    username: String! @id @search(by: [hash, regexp])
    reviews: [Review] @hasInverse(field: by)
}

 type Review {
    id: ID!
    about: Product!
    by: Customer!
    comment: String @search(by: [fulltext])
    rating: Int @search
}

 type Product {    productID: ID!    name: String @search(by: [term])    reviews: [Review] @hasInverse(field: about)} type Customer {    username: String! @id @search(by: [hash, regexp])    reviews: [Review] @hasInverse(field: by)} type Review {    id: ID!    about: Product!    by: Customer!    comment: String @search(by: [fulltext])    rating: Int @search}


mutation {
  updateGQLSchema( input: { set: { schema: " type Product {    productID: ID!    name: String @search(by: [term])    reviews: [Review] @hasInverse(field: about)} type Customer {    username: String! @id @search(by: [hash

                                                               'review(filter: '
                                                               'ReviewFilter, '
                                                               'order: '
                                                               'ReviewOrder, '
                                                               'first: Int, '
                                                               'offset: Int): '
                                                               '[Review]\n'
                                                               '\tmsg: String\n'
                                                               '\tnumUids: '
                                                               'Int\n'
                                                               '}\n'
                                                               '\n'
                                                               'type '
                

# Dgraph Python Clinet

- https://dgraph.io/docs/clients/python/

## Create client

In [22]:
client_stub = pydgraph.DgraphClientStub('localhost:{port}'.format(port=PORT_ALPHA_GRPC))
client = pydgraph.DgraphClient(client_stub)
dir(client)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_clients',
 '_common_except_alter',
 '_jwt',
 '_login_metadata',
 'add_login_metadata',
 'alter',
 'any_client',
 'async_alter',
 'check_version',
 'handle_alter_future',
 'login',
 'retry_login',
 'txn']

## Add schema

In [11]:
schema = """name: string @index(exact) ."""

op = pydgraph.Operation(schema=schema)
client.alter(op)

## Mutation

In [13]:
p = {
    'name': 'Alice',
}

txn = client.txn()
try:
    txn.mutate(set_obj=p)
    txn.commit()
finally:
    txn.discard()

## Query

In [15]:
query = """query all($a: string)
{
    all(func: eq(name, $a))
    {
        uid
    }
}"""
variables = {'$a': 'Alice'}

txn = client.txn(read_only=True)
try:
    res = txn.query(query,variables=variables)
    ppl = json.loads(res.json)
    pprint.pprint(ppl)
finally:
    txn.discard()

{'all': [{'uid': '0x2712'}]}


## Upsert

In [17]:
query = """{
    u as var(func: eq(name, "Alice"))
}"""
nquad = """
    uid(u) <name> "Alice" .
    uid(u) <age> "25" .
"""

txn = client.txn()
try:
    mutation = txn.create_mutation(set_nquads=nquad)
    request = txn.create_request(query=query,
                                 mutations=[mutation],
                                 commit_now=True)
    txn.do_request(request)
finally:
    txn.discard()

### Conditional upsert

In [23]:
query = """{
    user as var(func: eq(email, "wrong_email@dgraph.io"))
}"""
cond = "@if(eq(len(user), 1))"
nquads = """
    uid(user) <email> "correct_email@dgraph.io" .
"""

txn = client.txn()
try:
    mutation = txn.create_mutation(cond=cond, set_nquads=nquads)
    request = txn.create_request(mutations=[mutation],
                                 query=query,
                                 commit_now=True)
    txn.do_request(request)
finally:
    txn.discard()

## Close resource

In [20]:
client_stub.close()